# Classifiy toxicity and then rank toxicity

This kernel is inspired by Rhodium Beng Classifying multi-label comments with Logistic Regression made during the contest Jigsaw classification challenge.

The idea is to first make a classifier predicting the different toxicity and then use the predictions to make a score of toxicity.

The model used for classification is ClassifierChains with RandomForest.

The scoring function is toxic + obscene + insult + 1.5 * threat + 2 * severe_toxic + 2 * identity_hate

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
import string
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
import re
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
stop = stopwords.words('english')
punctuation = string.punctuation

## Load training and test data

In [ ]:
df_train = pd.read_csv(
    "../input/jigsaw-toxic-comment-classification-challenge/train.csv")

In [ ]:
df_test = pd.read_csv(
    "../input/jigsaw-toxic-comment-classification-challenge/test.csv")

In [ ]:
cols_target = ['toxic', 'obscene', 'insult',
               'threat', 'severe_toxic', 'identity_hate']

## Clean up the comment text

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
def remove_stopwords_sentence(sentence):
    # print(sentence)
    return pd.Series([word for word in sentence[0].split() if word not in stop])


def remove_stopwords_df(df):
    return df.apply(remove_stopwords_sentence, axis=1)


def stem_sentence(s):
    stemmer = EnglishStemmer()
    return pd.Series([stemmer.stem(w) for w in s if not pd.isna(w)]).to_frame().apply(' '.join, axis=0)[0]

In [ ]:
prep_pipeline = Pipeline(steps=[
    ('remove \n', FunctionTransformer(pd.DataFrame.replace, kw_args={
     'to_replace': '\n', 'value': ' ', 'regex': True}, validate=False)),
    ('remove numbers', FunctionTransformer(pd.DataFrame.replace, kw_args={
     'to_replace': '\d', 'value': '', 'regex': True}, validate=False)),
    ('remove html tags', FunctionTransformer(pd.DataFrame.replace,
                                             kw_args={'to_replace': '<.*?>', 'value': '', 'regex': True}, validate=False)),
    ('lower', FunctionTransformer(lambda x: x.squeeze(
        axis=1).str.lower().to_frame(), validate=False)),
    ('remove punctuation', FunctionTransformer(lambda x: x.squeeze(
        axis=1).str.replace('[{}]'.format(punctuation), '').to_frame(), validate=False)),
    ('remove stopwords', FunctionTransformer(remove_stopwords_df, validate=False)),
    ('stemming', FunctionTransformer(pd.DataFrame.apply, kw_args={
     'func': stem_sentence, 'axis': 1}, validate=False)),
    #('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('vectorizer', TfidfVectorizer(lowercase=False, analyzer='word', ngram_range=(1, 3), min_df=5,
                                   preprocessor=None, tokenizer=lambda i: str(i).split()))
])

In [ ]:
# clean the comment_text in train_df
df_train['comment_text'] = df_train['comment_text'].apply(
    lambda com: clean_text(com))

In [ ]:
# clean the comment_text in test_df
df_test['comment_text'] = df_test['comment_text'].map(
    lambda com: clean_text(com))


## Define X, y

In [ ]:
X = df_train.comment_text
test_X = df_test.comment_text

In [ ]:
print(X.shape, test_X.shape)

In [ ]:
y = df_train[cols_target]

In [ ]:
y_test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv')

In [ ]:
df_test = df_test.merge(y_test, on='id', how='left')

In [ ]:
df_test2 = df_test[df_test.toxic != -1]

## Pipeline

In [ ]:
# define the pipeline
pipeline = Pipeline(
    [
        ('prep', prep_pipeline),
        ("cc_rf", ClassifierChain(RandomForestClassifier(
            max_depth=50, min_samples_split=5), order='random', random_state=0)),
    ]
)

In [ ]:
# train 
pipeline.fit(X.to_frame().astype(str),y)

In [ ]:
# predict training data
pipe_pred_train = pipeline.predict(X.to_frame())

In [ ]:
# accuracy of training 
accuracy_score(y, pipe_pred_train)

In [ ]:
# accuracy of test
pipe_pred_test = pipeline.predict(df_test2.comment_text.to_frame())
accuracy_score(df_test2[cols_target], pipe_pred_test)

## Apply ranking

In [ ]:
validation_data = pd.read_csv(
    '../input/jigsaw-toxic-severity-rating/validation_data.csv')

In [ ]:
validation_data.head()

In [ ]:
# prepare validation data
validation_data['less_toxic'] = validation_data.less_toxic.apply(clean_text)
validation_data['more_toxic'] = validation_data.more_toxic.apply(clean_text)

In [ ]:
# predict proba of each class on validation data
val1_pred = pipeline.predict_proba(validation_data.less_toxic.to_frame())
val2_pred = pipeline.predict_proba(validation_data.more_toxic.to_frame())

In [ ]:
def score_function(a):
    a[3] = a[3]*1.5  # threat
    a[4] = a[4]*2  # severe_toxic
    a[5] = a[5]*2  # identity hate
    return a.sum()


# apply score function to predict proba
val1_pred_sum = np.apply_along_axis(score_function, axis=1, arr=val1_pred)
val2_pred_sum = np.apply_along_axis(score_function, axis=1, arr=val2_pred)

In [ ]:
# compute Validation score
(val1_pred_sum < val2_pred_sum).sum()/validation_data.shape[0]

# Comments to score

In [ ]:
comments_to_score = pd.read_csv(
    '../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
comments_to_score['text'] = comments_to_score.text.apply(clean_text)

In [ ]:
# predict on the comments to score
predictions = pipeline.predict_proba(comments_to_score.text.to_frame())

In [ ]:
# compute score of each comment
pred_sum = np.apply_along_axis(score_function,  axis=1, arr=predictions)

In [ ]:
pred_sum.shape

In [ ]:
# get submission template
sample_submission = pd.read_csv(
    '../input/jigsaw-toxic-severity-rating/sample_submission.csv')

In [ ]:
# add score to submissions
sample_submission['score'] = pred_sum

In [ ]:
# save submission
sample_submission.to_csv('./submission.csv', index=False)

Public leaderboard score = 0.769